In [2]:
import numpy as np
from numpy.polynomial import Polynomial
import timeit

def ls_american_option_quadratic_iter(X, t, r, strike):
    # At final time, payoff is just the European payoff
    cashflow = np.maximum(strike - X[-1, :], 0.0)
    
    # Go backwards from next-to-last time step down to the first
    for i in reversed(range(1, X.shape[0] - 1)):
        dt = (t[i+1] - t[i])
        df = np.exp(-r * dt)
        cashflow = cashflow * df
        
        x = X[i, :]
        exercise = np.maximum(strike - x, 0.0)
        itm = (exercise > 0)
        
        # Fit a 2nd-degree polynomial on the in-the-money subset
        fitted = Polynomial.fit(x[itm], cashflow[itm], deg=2)
        continuation = fitted(x)
        
        ex_idx = itm & (exercise > continuation)
        cashflow[ex_idx] = exercise[ex_idx]
        
        yield cashflow, x, fitted, continuation, exercise, ex_idx

def longstaff_schwartz_american_option_quadratic(X, t, r, strike):
    gen = ls_american_option_quadratic_iter(X, t, r, strike)
    cashflow = None
    for c, *_ in gen:
        cashflow = c
    if cashflow is None:
        raise ValueError("No cashflow computed in the iteration!")
    
    dt = (t[1] - t[0])
    return cashflow.mean() * np.exp(-r * dt)

def simulate_geometric_brownian_paths(S0, r, sigma, T, N, n_steps, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    dt = T / n_steps
    S = np.zeros((n_steps + 1, N))
    S[0, :] = S0
    
    for i in range(n_steps):
        z = np.random.normal(size=N)
        S[i+1, :] = S[i, :] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    
    return S

def run_pricing():
    # Parameters
    S0     = 36.0
    K      = 40.0
    r      = 0.06
    sigma  = 0.20
    T      = 2.0
    n_steps = 50
    N       = 100000  # Note: This might take a moment; you can reduce it for testing
    
    t = np.linspace(0, T, n_steps + 1)
    X = simulate_geometric_brownian_paths(S0, r, sigma, T, N, n_steps, seed=52)
    price = longstaff_schwartz_american_option_quadratic(X, t, r, K)
    return price

if __name__ == "__main__":
    # Run once to verify output
    price = run_pricing()
    print(f"American put via LSM Quadratic = {price:.4f}")
    
    # Measure execution time over 10 runs
    execution_time = timeit.timeit('run_pricing()', setup='from __main__ import run_pricing', number=10)
    print(f"Average execution time over 10 runs: {execution_time / 10:.4f} seconds")




American put via LSM Quadratic = 4.8184
Average execution time over 10 runs: 0.7971 seconds
